# Calculate the Indian Ocean Dipole (IOD)

Edited version of Jenna_test.ipynb

In [1]:
# imports
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# from warnings import filterwarnings
# filterwarnings('ignore')

In [5]:
# change this from MFdataset
# sst = xr.open_mfdataset('data_dir/sst-data/sorted_by_date/sst1982_2022.nc')['sst']
sst = xr.open_mfdataset('/home/jovyan/shared-public/Hesperosaurus_Bon_Fortepiano/data_dir/sst-data/sorted_by_date/sst1981_2023.nc')['sst']
sst

<xarray.DataArray 'sst' (time: 15399, lat: 720, lon: 1440)>
dask.array<open_dataset-sst, shape=(15399, 720, 1440), dtype=float32, chunksize=(15399, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2023-10-29
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  34.82]

In [4]:
pwd

'/home/jovyan/shared-public/Hesperosaurus_Bon_Fortepiano/James'

In [6]:
# select just the IOD regions (note our longitude values are in degrees east) by slicing
sst_West = sst.sel(lat=slice(-10, 10), lon=slice(50, 70))
sst_East = sst.sel(lat=slice(-10, 0), lon=slice(90, 110))
sst_West

<xarray.DataArray 'sst' (time: 15399, lat: 80, lon: 80)>
dask.array<getitem, shape=(15399, 80, 80), dtype=float32, chunksize=(15399, 80, 80), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2023-10-29
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
  * lon      (lon) float32 50.12 50.38 50.62 50.88 ... 69.12 69.38 69.62 69.88
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  34.82]

In [7]:
# select just the IOD regions (note our longitude values are in degrees east) by slicing
sst_West = sst.sel(lat=slice(-10, 10), lon=slice(50, 70))
sst_East = sst.sel(lat=slice(-10, 0), lon=slice(90, 110))
sst_East 

<xarray.DataArray 'sst' (time: 15399, lat: 40, lon: 80)>
dask.array<getitem, shape=(15399, 40, 80), dtype=float32, chunksize=(15399, 40, 80), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2023-10-29
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... -0.625 -0.375 -0.125
  * lon      (lon) float32 90.12 90.38 90.62 90.88 ... 109.1 109.4 109.6 109.9
Attributes:
    long_name:     Daily Sea Surface Temperature
    units:         degC
    valid_range:   [-3. 45.]
    precision:     2.0
    dataset:       NOAA High-resolution Blended Analysis
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Individual Observations
    actual_range:  [-1.8  34.82]

In [8]:
%%time

# find the monthly climatology
sst_clim_East = sst_East.groupby("time.month").mean(dim="time")
sst_clim_West = sst_West.groupby("time.month").mean(dim="time")

# find the monthly anomaly
sst_anom_East = sst_East.groupby("time.month") - sst_clim_East
sst_anom_West = sst_West.groupby("time.month") - sst_clim_West
sst_anom_West

CPU times: user 96.1 ms, sys: 7 ms, total: 103 ms
Wall time: 116 ms


<xarray.DataArray 'sst' (time: 15399, lat: 80, lon: 80)>
dask.array<sub, shape=(15399, 80, 80), dtype=float32, chunksize=(15399, 80, 80), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2023-10-29
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
  * lon      (lon) float32 50.12 50.38 50.62 50.88 ... 69.12 69.38 69.62 69.88
    month    (time) int64 9 9 9 9 9 9 9 9 9 9 ... 10 10 10 10 10 10 10 10 10 10

In [9]:
# note this takes a little while, it stays on 0% for a while then speeds up
from dask.diagnostics import ProgressBar

# this is Jenna Pearson's " .load() call "
with ProgressBar():
    sst_anom_West.load() # converts dask array to xarray dataset and loads into memory

[########################################] | 100% Completed | 435.59 s


In [ ]:
"""
13 Nov 2023: Jenna Pearson on Discord:
The  main issue I see with your notebook so far is trying
to blend CESM2's model grid with observational SST to get a weighted mean. 
When using observational data, to find a weighted mean please use the tools 
from the remote sensing W1D3 (see tutorial 5 for example). 
I have created a notebook in the same directory called Jenna_test.ipynb. 
Please take a look there for some example code.

Second,  I get the same crash you describe when trying to plot. 

I believe the issue is that when using xr.open_mfdataset to open the sst data, 
it is automatically stored as a dask array. 
Plotting and calling .to_netcdf() triggers a .compute/load() call 
which I believe is what is leading to all the 
memory problems.
I have placed a .load() call before doing the area weighted means 
which seems to work, and the data is small enough to manage outside of dask.
See screenshot for output. 

I might be quite slow to respond over the next couple weeks with travel and 
holidays coming up, but please feel free to reach out if this doesn't solve your issue.
@Ohad Zivan might also be able to jump online in my absence if he has time.

I hope this helps! Happy coding.
"""

In [10]:
# note this takes a little while
from dask.diagnostics import ProgressBar

# this is Jenna Pearson's " .load() call "
with ProgressBar():
    sst_anom_East.load()

[########################################] | 100% Completed | 400.96 s


In [11]:
# calculate the weights using the latitude coordinates
weights_West = np.cos(np.deg2rad(sst_anom_West.lat))
weights_West.name = "weights"
weights_West

# weights_East = np.cos(np.deg2rad(sst_anom_East.lat))
# weights_East.name = "weights"
# weights_East  # note these weights are VERY close to 1 
# as we are near the equator where the grid cells are similar in size

<xarray.DataArray 'weights' (lat: 80)>
array([0.98518425, 0.9859232 , 0.9866433 , 0.9873447 , 0.9880273 ,
       0.98869103, 0.98933595, 0.9899621 , 0.99056935, 0.9911577 ,
       0.9917273 , 0.9922779 , 0.99280965, 0.9933225 , 0.99381644,
       0.9942915 , 0.9947476 , 0.9951847 , 0.9956029 , 0.9960022 ,
       0.9963825 , 0.9967438 , 0.9970861 , 0.99740946, 0.99771386,
       0.99799925, 0.9982656 , 0.998513  , 0.9987413 , 0.99895066,
       0.999141  , 0.9993123 , 0.9994646 , 0.99959785, 0.99971205,
       0.99980724, 0.9998834 , 0.9999405 , 0.9999786 , 0.9999976 ,
       0.9999976 , 0.9999786 , 0.9999405 , 0.9998834 , 0.99980724,
       0.99971205, 0.99959785, 0.9994646 , 0.9993123 , 0.999141  ,
       0.99895066, 0.9987413 , 0.998513  , 0.9982656 , 0.99799925,
       0.99771386, 0.99740946, 0.9970861 , 0.9967438 , 0.9963825 ,
       0.9960022 , 0.9956029 , 0.9951847 , 0.9947476 , 0.9942915 ,
       0.99381644, 0.9933225 , 0.99280965, 0.9922779 , 0.9917273 ,
       0.9911577 , 0.99056935, 0.9899621 , 0.98933595, 0.98869103,
       0.9880273 , 0.9873447 , 0.9866433 , 0.9859232 , 0.98518425],
      dtype=float32)
Coordinates:
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
Attributes:
    long_name:      Latitude
    standard_name:  latitude
    units:          degrees_north
    actual_range:   [-89.875  89.875]
    axis:           Y

In [12]:
# calculate the weights using the latitude coordinates
# weights_West = np.cos(np.deg2rad(sst_anom_West.lat))
# weights_West.name = "weights"
# # weights_West

weights_East = np.cos(np.deg2rad(sst_anom_East.lat))
weights_East.name = "weights"
weights_East  # note these weights are VERY close to 1 
# as we are near the equator where the grid cells are similar in size

<xarray.DataArray 'weights' (lat: 40)>
array([0.98518425, 0.9859232 , 0.9866433 , 0.9873447 , 0.9880273 ,
       0.98869103, 0.98933595, 0.9899621 , 0.99056935, 0.9911577 ,
       0.9917273 , 0.9922779 , 0.99280965, 0.9933225 , 0.99381644,
       0.9942915 , 0.9947476 , 0.9951847 , 0.9956029 , 0.9960022 ,
       0.9963825 , 0.9967438 , 0.9970861 , 0.99740946, 0.99771386,
       0.99799925, 0.9982656 , 0.998513  , 0.9987413 , 0.99895066,
       0.999141  , 0.9993123 , 0.9994646 , 0.99959785, 0.99971205,
       0.99980724, 0.9998834 , 0.9999405 , 0.9999786 , 0.9999976 ],
      dtype=float32)
Coordinates:
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... -0.625 -0.375 -0.125
Attributes:
    long_name:      Latitude
    standard_name:  latitude
    units:          degrees_north
    actual_range:   [-89.875  89.875]
    axis:           Y

In [13]:
# take area weighted anomalies
sst_anom_mean_West = sst_anom_West.weighted(weights_West).mean(
    dim=["lat", "lon"]
)

sst_anom_mean_East = sst_anom_East.weighted(weights_East).mean(
    dim=["lat", "lon"]
)

In [14]:
# subtract two regions
sst_WestMinusEast_anom_mean = sst_anom_mean_West - sst_anom_mean_East
sst_WestMinusEast_anom_mean

<xarray.DataArray 'sst' (time: 15399)>
array([-0.33032686, -0.5516781 , -0.5238872 , ...,  1.5855153 ,
        1.512733  ,  1.2074288 ], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2023-10-29
    month    (time) int64 9 9 9 9 9 9 9 9 9 9 ... 10 10 10 10 10 10 10 10 10 10

In [16]:
#test plot of positive IOD (West minus East)
sst_WestMinusEast_anom_mean.plot()